In [6]:
pip install dash pandas plotly dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 8.1 MB/s eta 0:00:00


In [20]:
import dash
from dash import dcc, html, dash_table, Input, Output
import pandas as pd
import plotly.express as px
import random
import dash_bootstrap_components as dbc

# Sample Data Generation
vendors = ["Vendor A", "Vendor B", "Vendor C", "Vendor D"]
hospitals = ["Hospital X", "Hospital Y", "Hospital Z"]
drugs = ["Paracetamol", "Ibuprofen", "Amoxicillin", "Aspirin"]

data = {
    "Vendor": [random.choice(vendors) for _ in range(100)],
    "Drug": [random.choice(drugs) for _ in range(100)],
    "Quantity": [random.randint(100, 1000) for _ in range(100)],
    "Hospital": [random.choice(hospitals) for _ in range(100)],
    "Status": [random.choice(["Ordered", "Shipped", "Delivered"]) for _ in range(100)]
}

df = pd.DataFrame(data)

# Initialize Dash App with a Futuristic Theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

CARD_STYLE = {
    "background": "linear-gradient(145deg, #0f2027, #203a43, #2c5364)",
    "borderRadius": "20px",
    "padding": "25px",
    "boxShadow": "0px 5px 15px rgba(0,0,0,0.4)",
    "color": "white",
    "textAlign": "center",
    "transition": "transform 0.2s",
}

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("🚀 Smart Healthcare Procurement Dashboard", className="text-center text-white my-4"))
    ]),

    dbc.Row([
        dbc.Col(dcc.Dropdown(id="vendor_filter", options=[{"label": v, "value": v} for v in vendors], placeholder="🏢 Select Vendor", multi=True), width=4),
        dbc.Col(dcc.Dropdown(id="hospital_filter", options=[{"label": h, "value": h} for h in hospitals], placeholder="🏥 Select Hospital", multi=True), width=4),
        dbc.Col(dcc.Dropdown(id="status_filter", options=[{"label": s, "value": s} for s in ["Ordered", "Shipped", "Delivered"]], placeholder="✉️ Select Status", multi=True), width=4)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(html.Div(id="total_orders", style=CARD_STYLE), width=3),
        dbc.Col(html.Div(id="total_shipped", style=CARD_STYLE), width=3),
        dbc.Col(html.Div(id="total_delivered", style=CARD_STYLE), width=3),
        dbc.Col(html.Div(id="total_drugs", style=CARD_STYLE), width=3)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(id="shipment_pie"), width=6),
        dbc.Col(dcc.Graph(id="drug_bar"), width=6)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([
            html.H4("📅 Vendor Activity Log", className="text-white mb-3"),
            dash_table.DataTable(id="data_table",
                                 columns=[{"name": i, "id": i} for i in df.columns],
                                 style_table={'overflowX': 'auto'},
                                 style_header={'backgroundColor': 'black', 'color': 'white', 'fontWeight': 'bold'},
                                 style_cell={'backgroundColor': '#1e1e1e', 'color': 'white'})
        ])
    ])
], fluid=True)

@app.callback(
    [Output("shipment_pie", "figure"), Output("drug_bar", "figure"),
     Output("data_table", "data"),
     Output("total_orders", "children"), Output("total_shipped", "children"),
     Output("total_delivered", "children"), Output("total_drugs", "children")],
    [Input("vendor_filter", "value"), Input("hospital_filter", "value"), Input("status_filter", "value")]
)
def update_dashboard(vendors_selected, hospitals_selected, statuses_selected):
    filtered = df.copy()
    if vendors_selected:
        filtered = filtered[filtered['Vendor'].isin(vendors_selected)]
    if hospitals_selected:
        filtered = filtered[filtered['Hospital'].isin(hospitals_selected)]
    if statuses_selected:
        filtered = filtered[filtered['Status'].isin(statuses_selected)]

    pie = px.pie(filtered, names='Status', title="📈 Distribution Status", color_discrete_sequence=px.colors.sequential.Plasma)
    bar = px.bar(filtered.groupby('Drug')['Quantity'].sum().reset_index(), x='Drug', y='Quantity',
                 title="🔹 Drug Usage", color='Drug', color_discrete_sequence=px.colors.sequential.Agsunset)

    return pie, bar, filtered.to_dict("records"), \
           f"📦 Total Orders: {len(filtered)}", \
           f"🚚 Shipped: {len(filtered[filtered['Status'] == 'Shipped'])}", \
           f"✅ Delivered: {len(filtered[filtered['Status'] == 'Delivered'])}", \
           f"💊 Unique Drugs: {filtered['Drug'].nunique()}"

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>